In [ ]:
import numpy as np
import math
import scipy.linalg as la
import matplotlib.pyplot as plt
import scipy
from mpl_toolkits.mplot3d import Axes3D


## Task 1.1 

In [ ]:
def eulerstep(Tdx, uold, dt):  ##My guy gör comeback 
    unew = uold + dt*(Tdx@uold)
    return unew

In [ ]:
def eulersolve(g0, alpha, beta, tend, N, M):
     deltaX = 1/(N+2)  # rumssteg från x0 till xN+1
     dt = tend/M  # tidsteg från t0 till tN
     Xgrid = np.linspace(0, 1, N+2)
     tgrid = np.linspace(0, tend, M+1)
     Tdx = la.toeplitz(np.array([-2, 1] + [0] * (N-2))) ##Skapar en Toeplitz matris för disc av andraderivatan
     Tdx = Tdx/(deltaX**2)   #Den hanterar endast interior grid points 

     Y = np.zeros((M+1, N)) #Första värdet är tidssteget, andra är x-steget.
     Y[0,:] = g0

     for i in range(M):
          Y[i+1,:] = np.transpose(eulerstep(Tdx, np.transpose(Y[i, :]), dt))
     Y = np.hstack((alpha*np.ones((M+1, 1)), Y, beta*np.ones((M+1, 1)))) #Lägger till gränsvärdena
     
     return Y, Xgrid, tgrid
          


In [ ]:
def plot3D(Y, Xgrid, tgrid):
     T, X = np.meshgrid(np.transpose(tgrid), Xgrid)

     # Create a 3D plot
     fig = plt.figure(figsize=(10, 10))  # You can adjust the size as needed
     ax = fig.add_subplot(111, projection='3d')
     ax.plot_surface(T, X, np.transpose(Y))

     ax.set_xlabel('Time')
     ax.set_ylabel('Space')
     ax.set_zlabel('Value')

     plt.show()

In [ ]:
N = 10
M = 1000
if M > 2*(N+1)**2:
     print("will converge")
else:
     print("This will not converge")
alpha = 0
beta = 0
g0 = np.random.normal(loc=0.0, scale=1.0, size=N)
tend = 1
Y, Xgrid, tgrid = eulersolve(g0, alpha, beta, tend, N, M)
plot3D(Y, Xgrid, tgrid)


In [ ]:
N = 10
alpha = 0
beta = 0
g0 = np.random.normal(loc=0.0, scale=1.0, size=N)
tend = 1
maxX = 0
for X in range(1, 100):
     X = X/100
     M = int(1/X*(N+1)**2)
     Y, Xgrid, tgrid = eulersolve(g0, alpha, beta, tend, N, M)
     if np.sum(np.abs(Y[-1,:])) < 0.001:
          #print('Converged at X = ' + str(X))
          maxX = X
     else:
          #print('Did not converge at X = ' +  str(X))
          break
#plot3D(Y, Xgrid, tgrid)

print('CFL = ' + str(maxX))